In [ ]:
!pip install langchain rank_bm25 pypdf unstructured chromadb
!pip install unstructured['pdf'] unstructured
!apt-get install poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract

### Load the required Packages

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub


from langchain.retrievers import BM25Retriever, EnsembleRetriever

import os

In [ ]:
### Load the PDF file

In [ ]:
file_path = "./data/Orca_paper.pdf"
data_file = UnstructuredPDFLoader(file_path)
docs = data_file.load()

In [ ]:
print(docs[0].page_content)

### Split Documents and Chunking

In [ ]:
# create chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                          chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [ ]:
chunks[0].page_content

'3 2 0 2\n\nn u J\n\n5\n\n] L C . s c [\n\n1 v 7 0 7 2 0 . 6 0 3 2 : v i X r a\n\nOrca: Progressive Learning from Complex\n\nExplanation Traces of GPT-4\n\nSubhabrata Mukherjee∗†, Arindam Mitra∗\n\nGanesh Jawahar, Sahaj Agarwal, Hamid Palangi, Ahmed Awadallah\n\nMicrosoft Research\n\nAbstract'

In [ ]:
# Get Embedding Model from HF via API

from google.colab import userdata
HF_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN, model_name="BAAI/bge-base-en-v1.5"
)

### VectorStore

In [ ]:
# Vector store with the selected embedding model
vectorstore = Chroma.from_documents(chunks, embeddings)

In [ ]:
vectorstore_retreiver = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k =  3

### Ensemble Retriever

In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,
                                                   keyword_retriever],
                                       weights=[0.5, 0.5])

In [ ]:
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    model_kwargs={"temperature": 0.3,"max_new_tokens":1024},
    huggingfacehub_api_token=HF_TOKEN,
)

### Prompt Template:

In [ ]:
template = """
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

In [ ]:
chain = (
    {"context": ensemble_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [ ]:
print(chain.invoke("What is instruction tuning?"))

Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question. 

Think step by step before answering the question. You will get a $100 tip if you provide correct answer. 

CONTEXT: [Document(page_content='2 Preliminaries\n\n2.1\n\nInstruction Tuning\n\nInstruction tuning [22] is a technique that allows pre-trained language models to learn from input (natural language descriptions of the task) and response pairs, for example, {"instruction": correct sentence.", "input": fox jumped quickly"}. Instruction tuning has been applied to both language-only and multimodal tasks. For language-only tasks, instruction tuning has been shown to improve the zero-shot and few-shot performance of models such as FLAN [22] and InstructGPT [5] on various benchmarks. For multimodal tasks, instruction tuning has been used to generate synthetic instruction-following data for language-image tasks, such as image captioning [23] and

Instruction tuning is a technique used to train pre-trained language models to learn from input (natural language descriptions of the task) and response pairs. This technique allows the models to better understand and follow instructions given to them, improving their performance on various benchmarks for both language-only and multimodal tasks. For language-only tasks, instruction tuning has been shown to improve the zero-shot and few-shot performance of models such as FLAN and InstructGPT on various

In [ ]:
print(chain.invoke("How does Orca compares to ChatGPT?"))

Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question. 

Think step by step before answering the question. You will get a $100 tip if you provide correct answer. 

CONTEXT: [Document(page_content='While significantly better than Vicuna and marginally better than ChatGPT, Orca’s average performance of 49.7%, lags GPT-4 by 26%. Note that GPT-4 has reported a data contami- nation issue with Big-Bench and that we are not aware of such issues with either LLaMA’s training data (the base model used by both Vicuna and Orca) or the Flan-V2 collection or Vicuna’s training data (ShareGPT).\n\nGiven the close performance on average on BigBench-Hard, we take a deeper look at differences in performance between Orca and ChatGPT:\n\nEntailment and Semantic Understanding:\n\nOrca performs better at entailment (formal fallacies) and semantic understanding (Dis- ambiguation QA and Snarks).', metadata={'source': './da